In [105]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse

In [94]:
url_base='http://www.csmpt.org.tw/news'
url_work_table='/index.php?type=4'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [95]:
pages_link=[]
for o in soup.find(class_="page").find_all('span'):
    if o.find('a'):
        O=o.find("a")
        if O.text == "到最後>|" or O.text == ">":
            pass
        else:
            link = url.replace("?type=4","")+O.get('href')
            pages_link.append(link)
            print(O.text,link)

2 http://www.csmpt.org.tw/news/index.php?sub=2&continue=Y&type=4
3 http://www.csmpt.org.tw/news/index.php?sub=3&continue=Y&type=4


In [84]:
http://www.csmpt.org.tw/news/index.php?sub=3&continue=Y&type=4

SyntaxError: invalid syntax (1276046040.py, line 1)

# Get work table

The `get_work_page` function is used for getting each work page

In [96]:
work_page = soup.find_all('article')[0]

In [8]:
def get_work_page(page_url,page=None,page_link_part='?page='):
    global url
    if page != None:
        page_url = page_url+page_link_part+str(page)
    if url == None:
        page_url = url
        print(page_url)
    g=requests.get(page_url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

Using `get_work_page` to read each page and using `get_tables_part` read work table

In [36]:
def get_work_detail(link):
    g=requests.get(link)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [249]:
def get_work_deadtime(link):
    y=get_work_detail(link).find('div', class_="content").text.replace('\xa0','').replace("\n","").replace(" 年","年").replace(" 月","月").replace(" 日","日")
    date_pattern="(\d{1,4}年)((([0?][1-9])月)|(([1?][0-2])月)|([1-9]月)?)(([0?][1-9]日)|([1?][0-9]日)|([2?][1-9]日)|([3][0-1]日)?)"
    return re.search(date_pattern,re.search("(?<=期限)(.*)年(.*)月(.*)日",y)[0]).group(0)

In [252]:
def get_work_table(table_,work_table):
    for p_i, p_item in enumerate(table_.find_all('li')):
        if p_item.find('a'):
            p_item_a = p_item.find('a')
            title = p_item_a.text
            link_s = url_base+'/'+p_item_a.get('href')
            try:
                dead_line = get_work_deadtime(link_s)
            except:
                dead_line = "null(check webpage)"
            print(p_i-1,title,link_s,dead_line)
            work_table.append([p_i-1, title, dead_line, link_s ])
    return work_table

In [253]:
work_table=[]
work_table = get_work_table(work_page,work_table)
for p_i, p_item in enumerate(pages_link):
    table_ = get_work_page(p_item)

    work_table = get_work_table(table_,work_table)


1 合富醫療集團誠徵資深物理師 http://www.csmpt.org.tw/news/content.php?id=341&type=4 null(check webpage)
2 高雄長庚醫院招募醫學物理師人員 http://www.csmpt.org.tw/news/content.php?id=340&type=4 2022年5月25日
3 Precess Medical Derivative http://www.csmpt.org.tw/news/content.php?id=333&type=4 null(check webpage)
4 聯新國際醫院放射腫瘤科 誠徵醫學物理師ㄧ名 http://www.csmpt.org.tw/news/content.php?id=330&type=4 null(check webpage)
5 久和醫療儀器股份有限公司誠徵醫學物理師 http://www.csmpt.org.tw/news/content.php?id=328&type=4 null(check webpage)
6 Global Standards and Content Physicist (物理師) http://www.csmpt.org.tw/news/content.php?id=320&type=4 null(check webpage)
7 Product Support Specialist (Radiation) http://www.csmpt.org.tw/news/content.php?id=305&type=4 null(check webpage)
8 衛生福利部臺北醫院  即將成立放射腫瘤科  誠徵約用醫學物理師一名 http://www.csmpt.org.tw/news/content.php?id=303&type=4 null(check webpage)
9 國立成功大學醫學院附設醫院腫瘤醫學部對外甄選醫學物理師一名 http://www.csmpt.org.tw/news/content.php?id=302&type=4 110年11月12日
10 清華大學原子科學院硼中子捕獲治療中心  誠徵  醫學物理師 http://www.csmpt.org.tw/news/content.php?id=2

In [254]:
work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','期限' ,'連結'])

print(work_table)

    no                                               召聘職稱  \
0    1                                      合富醫療集團誠徵資深物理師   
1    2                                    高雄長庚醫院招募醫學物理師人員   
2    3                         Precess Medical Derivative   
3    4                              聯新國際醫院放射腫瘤科 誠徵醫學物理師ㄧ名   
4    5                                久和醫療儀器股份有限公司誠徵醫學物理師   
5    6       Global Standards and Content Physicist (物理師)   
6    7             Product Support Specialist (Radiation)   
7    8                  衛生福利部臺北醫院  即將成立放射腫瘤科  誠徵約用醫學物理師一名   
8    9                      國立成功大學醫學院附設醫院腫瘤醫學部對外甄選醫學物理師一名   
9   10                      清華大學原子科學院硼中子捕獲治療中心  誠徵  醫學物理師   
10   1                            嘉義大林慈濟醫院放射腫瘤科 誠徵醫學物理師一名   
11   2                         中國附醫台中總院 放射腫瘤科 誠徴 醫學物理師 一位   
12   3                          臺北醫學大學附設醫院放射腫瘤科擴編，誠徵物理師3名   
13   4                國軍臺中總醫院附設民眾診療服務處 放射腫瘤科聘請民聘二等醫學物理師1員   
14   5                         亞洲大學附屬醫院 放射腫瘤科 誠徴 醫學物理師 一位   
15   6  Accuray 原廠徵 Clin